# Test json_crawler on BalanceSheet, General Ledger, and TransactionList reports

In [8]:
import pandas as pd
import json
import re

balance_sheet_path    = "C:\\Users\\paulcassidy\\OneDrive - Archdiocese of Chicago\\Documents\\JSON_Parsing\\RawData\\BalanceSheet.json"
transaction_list_path = "C:\\Users\\paulcassidy\\OneDrive - Archdiocese of Chicago\\Documents\\JSON_Parsing\\RawData\\TransactionList.json"
general_ledger_path   = "C:\\Users\\paulcassidy\\OneDrive - Archdiocese of Chicago\\Documents\\JSON_Parsing\\RawData\\GeneralLedger.json"

with open(balance_sheet_path) as f:
    balance_sheet = json.load(f)

with open(transaction_list_path) as f:
    transaction_list = json.load(f)

with open(general_ledger_path) as f:
    general_ledger = json.load(f)

### Balance sheet:
this one **worked**

In [4]:
final.json_to_csv(final.json_crawler(balance_sheet), "Output", "test_bs")

### Transaction List:
this one did **not** work

In [6]:
final.json_to_csv(final.json_crawler(transaction_list), "Output", "test_tl")

In [3]:
final.json_to_csv(final.json_crawler_dyno(transaction_list), "Output", "test_tl")

### General Ledger:
this one did **not** work

In [5]:
final.json_to_csv(final.json_crawler(general_ledger), "Output", "test_gl")

In [5]:
final.json_to_csv(final.json_crawler_dyno(general_ledger), "Output", "test_gl")

In [10]:
one = pd.json_normalize(general_ledger)
one

,Header.Time,Header.ReportName,Header.ReportBasis,Header.StartPeriod,Header.EndPeriod,Header.Currency,Header.Option,Columns.Column,Rows.Row
0,2021-05-14T14:04:16-07:00,GeneralLedger,Accrual,2021-01-01,2021-05-01,USD,"[{'Name': 'NoReportData', 'Value': 'false'}]","[{'ColTitle': 'Date', 'ColType': 'tx_date'}, {...",[{'Header': {'ColData': [{'value': '1100 Cash'...


In [ ]:
two = pd.json_normalize(one['Rows.Row'][0])
print(two)

In [36]:
pd.json_normalize(two['Header.ColData'][104])

,value,id
0,6110 Capital Improvements,661
1,,NaN
2,,NaN
3,,NaN
4,,NaN
5,,NaN
6,,NaN
7,,NaN


In [37]:
three = pd.json_normalize(two['Rows.Row'][0])
three

,type,Header.ColData,Rows.Row,Summary.ColData
0,Section,[{'value': '1101 Saints JFX Oper. Acct. #2264'...,"[{'ColData': [{'value': 'Beginning Balance'}, ...",[{'value': 'Total for 1101 Saints JFX Oper. Ac...
1,Section,"[{'value': '1101.10 Checking Available', 'id':...","[{'ColData': [{'value': 'Beginning Balance'}, ...",[{'value': 'Total for 1101.10 Checking Availab...
2,Section,"[{'value': '1101.20 Checking Restricted', 'id'...",[{'Header': {'ColData': [{'value': '1101.22 Bu...,[{'value': 'Total for 1101.24 AOC A/P Restrict...
3,Section,"[{'value': '1101.25 Church Restricted', 'id': ...",[{'Header': {'ColData': [{'value': '101.25A Sh...,[{'value': 'Total for 1101.25 Church Restricte...
4,Section,"[{'value': '1101.27 Athletics', 'id': '623'}, ...","[{'ColData': [{'value': 'Beginning Balance'}, ...","[{'value': 'Total for 1101.27 Athletics'}, {'v..."
5,Section,"[{'value': '1101.29 Haiti', 'id': '697'}, {'va...",[{'Header': {'ColData': [{'value': '101.29A Ha...,"[{'value': 'Total for 1101.29 Haiti'}, {'value..."
6,Section,"[{'value': '1101.30 Men's Club', 'id': '687'},...","[{'ColData': [{'value': 'Beginning Balance'}, ...","[{'value': 'Total for 1101.30 Men's Club'}, {'..."


In [12]:
pd.json_normalize(three['Header.ColData'][0])

,value
0,1101 Saints JFX Oper. Acct. #2264
1,
2,
3,
4,
5,
6,
7,


In [38]:
four = pd.json_normalize(three['Rows.Row'][0])
four

,ColData,type
0,"[{'value': 'Beginning Balance'}, {'value': ''}...",Data
1,"[{'value': '2021-01-19'}, {'value': 'Transfer'...",Data


In [51]:
pd.json_normalize(four['ColData'][1])
test = pd.json_normalize(four['ColData'][1])

In [52]:
test['value'][0]

'2021-01-19'

## **What to get from the GeneralLedger report**

|       Field Name       |    'Rows.Row' Lvl    |           Norm Arg           |
|:-----------------------|:--------------------:|-----------------------------:|
|AccountName             | 1                    | Header.ColData['value'][0]   |
|AccountID               | 1                    | Header.ColData['id'][0]      |
|SubAccountName          | 2                    | Header.ColData['value'][0]   |
|BeginningBalance        | 3                    | ColData['value'][7]          |
|TransactionDate         | 3                    | ColData['value'][0]          |
|TransactionType         | 3                    | ColData['value'][1]          |
|TransactionTypeID       | 3                    | ColData['id'][1]             |
|DocumentNumber          | 3                    | ColData['value'][2]          |
|CompanyName             | 3                    | ColData['value'][3]          |
|CompanyNameID           | 3                    | ColData['id'][3]             |
|Memo                    | 3                    | ColData['value'][4]          |
|SplitFlag               | 3                    | ColData['value'][5]          |
|SplitID                 | 3                    | ColData['id'][5]             |
|TransactionAmount       | 3                    | ColData['value'][6]          |
|NewAccountBalance       | 3                    | ColData['value'][7]          |

- **'Header.ColData'** will only ever have the Account/SubAccount Name and ID
    - Name: 'Header.ColData['value'][0]
    - ID: 'Header.ColData['id'][0]

In [79]:
def parse_record_gl(data):
    for row in range(data.shape[0]):
            an   = Header.ColData['value'][0]
            aid  = Header.ColData['id'][0]  
            san  = Header.ColData['value'][0] 
            bb   = ColData['value'][7]
            td   = ColData['value'][0]
            tt   = ColData['value'][1]
            ttid = ColData['id'][1]
            dn   = ColData['value'][2]
            cn   = ColData['value'][3]
            cnid = ColData['id'][3]
            m    = ColData['value'][4]
            sf   = ColData['value'][5]
            sid  = ColData['id'][5]
            ta   = ColData['value'][6]
            nab  = ColData['value'][7]
            if not acc_val:
                acc_val = "NA"
            return {
                'AccountName' : an,
                'AccountID' : aid,
                'SubAccountName' : san,
                'BeginningBalance' : bb,
                'TransactionDate' : td,
                'TransactionType' : tt,
                'TransactionTypeID' : ttid,
                'DocumentNumber' : dn,
                'CompanyName' : cn,
                'CompanyNameID' : cnid,
                'Memo' : m,
                'SplitFlag' : sf,
                'SplitID' : sid,
                'TransactionAmount' : ta,
                'NewAccountBalance' : nab
                }

def json_crawler(data):
    # If data is dictionary returned by json.load(), then flatten it to initial pandas data frame
    if isinstance(data, dict):
        data = pd.json_normalize(data)
        return_value = json_crawler(data)
        return return_value 
    
    # Crawl through JSON file, normalizing by different keys to find account data and save to lst
    else:
        # Initialize list to hold account data objects
        lst = []
        for row in range(data.shape[0]):
            # Normalize by 'ColData'
            try:
                record_data = pd.json_normalize(data['ColData'][row])
                lst.append(parse_record_static(record_data))
            except:
                pass
            # Normalize by 'Header.ColData'
            try:
                record_data = pd.json_normalize(data['Header.ColData'][row])
                lst.append(parse_record_static(record_data))
            except:
                pass
            # Normalize by 'Rows.Row'
            try:
                row_data = pd.json_normalize(data['Rows.Row'][row])
                # Capture lst and return value before recursive function call
                return_value = json_crawler(row_data)
                lst = lst + return_value
            except:
                pass
        return lst

In [6]:
def parse_record_gl_tl(data):
    """

    Iterate through ColData or Header.ColData data frame. 
    Check what kind of data each data frame contains.
    Parse each record in data frame accordingly and return values.

    """

    # regular expression matching date format
    rex = re.compile("^[0-9]{4}-[0-9]{2}-[0-9]{2}$")

    # check which ColData data frame is being parsed

    for row in range(data.shape[0]):
        # transaction detail data frame
        if re.search(rex, data['value'][0]):
            td   = data['value'][0]
            tt   = data['value'][1]
            ttid = data['id'][1]
            dn   = data['value'][2]
            vn   = data['value'][3]
            vid  = data['id'][3]
            m    = data['value'][4]
            s    = data['value'][5]
            sid  = data['id'][5]
            ta   = data['value'][6]
            nab  = data['value'][7]
            
            vals = {
                'TransactionDate' : td,
                'TransactionType' : tt,
                'TransactionTypeID' : ttid,
                'DocumentNumber' : dn,
                'VendorName' : vn,
                'VendorID' : vid,
                'Memo' : m,
                'Split' : s,
                'SplitID' : sid,
                'TransactionAmount' : ta,
                'NewAccountBalance' : nab
                }

            # replace missing values           
            for k in vals:
                if not vals[k]:
                    vals[k] = "NA"
            
            return vals

        # beginning balance data frame
        elif data['value'][0] == "Beginning Balance":
            bb   = ColData['value'][7]
            return {
                'BeginningBalance' : bb
            }

        # name data frame
        else:
            if 'id' in data:
                an  = data['value'][0]
                aid = data['id'][0]
            else:
                an  = data['value'][0]
                aid = "NA"
                
            return {
                'AccountName' : an,
                'AccountID' : aid
            } 

In [7]:
def json_crawler_gl_tl(data):
    # If data is dictionary returned by json.load(), then flatten it to initial pandas data frame
    if isinstance(data, dict):
        data = pd.json_normalize(data)
        return_value = json_crawler_gl_tl(data)
        return return_value 
    
    # Crawl through JSON file, normalizing by different keys to find account data and save to lst
    else:
        # Initialize list to hold account data objects
        lst = []
        for row in range(data.shape[0]):
            # Normalize by 'ColData'
            try:
                record_data = pd.json_normalize(data['ColData'][row])
                lst.append(parse_record_gl_tl(record_data))
                # print("normalized by ColData")
            except:
                # print("uh oh!")
                pass
            # Normalize by 'Header.ColData'
            try:
                record_data = pd.json_normalize(data['Header.ColData'][row])
                lst.append(parse_record_gl_tl(record_data))
                # print("normalized by Header.ColData")
            except:
                pass
            # Normalize by 'Rows.Row'
            try:
                row_data = pd.json_normalize(data['Rows.Row'][row])
                # Capture lst and return value before recursive function call
                return_value = json_crawler_gl_tl(row_data)
                lst = lst + return_value
            except:
                pass

        return lst

In [8]:
json_crawler_gl_tl(general_ledger)

[{'AccountName': '1100 Cash', 'AccountID': '565'},
 {'AccountName': '1101 Saints JFX Oper. Acct. #2264', 'AccountID': 'NA'},
 {'TransactionDate': '2021-01-19',
  'TransactionType': 'Transfer',
  'TransactionTypeID': '4505',
  'DocumentNumber': 'NA',
  'VendorName': 'NA',
  'VendorID': 'NA',
  'Memo': 'transfer debit',
  'Split': "1132.02 Cash:Auxiliary Accounts:St Joseph Auxiliary Accounts:St Joseph Men's Club #9953",
  'SplitID': '314',
  'TransactionAmount': '-22.83',
  'NewAccountBalance': '-13122.83'},
 {'AccountName': '1101.10 Checking Available', 'AccountID': '691'},
 {'TransactionDate': '2021-01-01',
  'TransactionType': 'Journal Entry',
  'TransactionTypeID': '10581',
  'DocumentNumber': 'GC 210101',
  'VendorName': 'NA',
  'VendorID': 'NA',
  'Memo': 'NA',
  'Split': '-Split-',
  'SplitID': 'NA',
  'TransactionAmount': '400.00',
  'NewAccountBalance': '18750.75'},
 {'TransactionDate': '2021-01-01',
  'TransactionType': 'Journal Entry',
  'TransactionTypeID': '10581',
  'Docume

In [74]:
rex = re.compile("^[0-9]{4}-[0-9]{2}-[0-9]{2}$")
somestring = test['value'][0]

re.search(rex, somestring)

if re.search(rex, somestring):
    print("True")
else:
    print("False")

True


In [92]:
vals = {
                'TransactionDate' : "",
                'TransactionType' : "",
                'TransactionTypeID' : "",
                'DocumentNumber' : "",
                'CompanyName' : "",
                'CompanyNameID' : "",
                'Memo' : "",
                'SplitFlag' : "",
                'SplitID' : "",
                'TransactionAmount' : "",
                'NewAccountBalance' : ""
                }

# for k in range(len(vals)):
#                 if not vals[k]:
#                     vals[k] = "NA"

print(vals["TransactionType"])

In [95]:
for k in vals:
    print(k)

TransactionDate
TransactionType
TransactionTypeID
DocumentNumber
CompanyName
CompanyNameID
Memo
SplitFlag
SplitID
TransactionAmount
NewAccountBalance
